In [1]:
import os
import math
import pandas as pd
import time

In [2]:
os.listdir ("./data")

['created',
 'First and Future Data Manual 12062018.docx',
 'game_data.csv',
 'NGS-2016-post.csv',
 'NGS-2016-pre.csv',
 'NGS-2016-reg-wk1-6.csv',
 'NGS-2016-reg-wk13-17.csv',
 'NGS-2016-reg-wk7-12.csv',
 'NGS-2017-post.csv',
 'NGS-2017-pre.csv',
 'NGS-2017-reg-wk1-6.csv',
 'NGS-2017-reg-wk13-17.csv',
 'NGS-2017-reg-wk7-12.csv',
 'player_punt_data.csv',
 'play_information.csv',
 'play_player_role_data.csv',
 'video_footage-control.csv',
 'video_footage-injury.csv',
 'video_review.csv',
 '~$rst and Future Data Manual 12062018.docx']

In [3]:
df = pd.read_csv ("./data/NGS-2017-reg-wk1-6.csv", parse_dates = ['Time'])
df.sort_values (['GameKey', 'PlayID', 'GSISID', 'Time'], inplace = True)
df.head(5)

,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event
5144770,2017,399,1336,23449.0,2017-09-08 01:31:11.000,41.560001,54.279999,0.13,78.879997,192.050003,NaN
5144771,2017,399,1336,23449.0,2017-09-08 01:31:11.100,41.540001,54.139999,0.14,77.760002,190.759995,NaN
5144772,2017,399,1336,23449.0,2017-09-08 01:31:11.200,41.509998,54.000000,0.14,76.459999,189.559998,NaN
5144773,2017,399,1336,23449.0,2017-09-08 01:31:11.300,41.490002,53.860001,0.14,75.800003,188.449997,NaN
5144320,2017,399,1336,23449.0,2017-09-08 01:31:11.400,41.470001,53.709999,0.15,75.769997,187.589996,NaN


In [4]:
groupe = df.groupby (['Season_Year','GameKey', 'PlayID', 'GSISID'])["Time","Event"].count()
groupe.columns = ['Time_Player', 'Nb_event_played']
groupe.reset_index(inplace=True)
groupe.head(5)

,Season_Year,GameKey,PlayID,GSISID,Time_Player,Nb_event_played
0,2017,399,1336,23449.0,29,1
1,2017,399,1336,23534.0,205,6
2,2017,399,1336,26035.0,205,6
3,2017,399,1336,27010.0,205,6
4,2017,399,1336,27442.0,205,6


In [5]:
groupy = groupe.groupby(["Season_Year","GameKey","PlayID"])['Time_Player',"Nb_event_played"].max()
groupy.columns = ["Duration","Max_number_event"]
groupy.reset_index(inplace=True)
groupy.head(5)

,Season_Year,GameKey,PlayID,Duration,Max_number_event
0,2017,399,1336,205,6
1,2017,399,1776,862,10
2,2017,399,1884,971,8
3,2017,399,2572,212,6
4,2017,399,2712,663,11


In [6]:
select_player = pd.merge(groupe, groupy, on = ["Season_Year","GameKey","PlayID"], how = "inner")
select_player.head(4)#The shape are checked to be sure of changes

,Season_Year,GameKey,PlayID,GSISID,Time_Player,Nb_event_played,Duration,Max_number_event
0,2017,399,1336,23449.0,29,1,205,6
1,2017,399,1336,23534.0,205,6,205,6
2,2017,399,1336,26035.0,205,6,205,6
3,2017,399,1336,27010.0,205,6,205,6


In [7]:
select_player = select_player.loc[(select_player.Time_Player == select_player.Duration )]
select_player.shape[0]

19758

In [8]:
test_22 = select_player.groupby(["Season_Year","GameKey","PlayID"])['GSISID'].count()
print(test_22.shape)
print(test_22.loc[test_22==22].shape)

(907,)
(853,)


We realize that there might be another way of detecting which player is involved in the action. 
Using the player role data csv


In [9]:
"""The events are a vital information, let's understand them better"""
set(df['Event'])

{nan,
 'run',
 'pass_outcome_interception',
 'punt_land',
 'timeout_tv',
 'punt',
 'pass_outcome_incomplete',
 'fair_catch',
 'kick_received',
 'pass_outcome_caught',
 'play_submit',
 'punt_downed',
 'man_in_motion',
 'snap_direct',
 'out_of_bounds',
 'penalty_declined',
 'shift',
 'punt_blocked',
 'first_contact',
 'touchback',
 'penalty_accepted',
 'timeout',
 'ball_snap',
 'huddle_start_offense',
 'penalty_flag',
 'fumble_defense_recovered',
 'line_set',
 'punt_received',
 'tackle',
 'pass_forward',
 'punt_muffed',
 'punt_play',
 'pass_arrived',
 'punt_fake',
 'lateral',
 'huddle_break_offense',
 'fumble_offense_recovered',
 'fumble',
 'touchdown'}

In [10]:
"""Let's analyse the plays and select only the punt plays to begin with"""
punt_play = pd.read_csv("./data/play_information.csv")
punt_play.head(2)

,Season_Year,Season_Type,GameKey,Game_Date,Week,PlayID,Game_Clock,YardLine,Quarter,Play_Type,Poss_Team,Home_Team_Visit_Team,Score_Home_Visiting,PlayDescription
0,2016,Pre,2,08/13/2016,2,191,12:30,LA 47,1,Punt,LA,LA-DAL,0 - 7,"(12:30) J.Hekker punts 52 yards to DAL 1, Cent..."
1,2016,Pre,2,08/13/2016,2,1132,12:08,LA 29,2,Punt,LA,LA-DAL,7 - 21,"(12:08) J.Hekker punts 51 yards to DAL 20, Cen..."


In [11]:
set(punt_play['Play_Type'])

{'Punt'}

It seems that all the plays in the info are puntplays.

In [12]:
simple_play_info = pd.DataFrame()
simple_play_info['Season_Year']=punt_play['Season_Year']
simple_play_info['GameKey'] = punt_play['GameKey']
simple_play_info['PlayID'] = punt_play['PlayID']
simple_play_info.head(5)


,Season_Year,GameKey,PlayID
0,2016,2,191
1,2016,2,1132
2,2016,2,1227
3,2016,2,1864
4,2016,2,2247


In [13]:
merged_plays = pd.merge(df, simple_play_info, on = ["Season_Year","GameKey","PlayID"], how = "inner")

In [14]:
print(merged_plays.shape)
df.shape

(9422296, 11)


(9433922, 11)

There is not a major difference between the two so we guess that the vaste majority of plays in the statistics dataframe are already punt plays.

In [15]:
new_merged_plays =  pd.merge(merged_plays, groupy, on = ["Season_Year","GameKey","PlayID"], how = "inner")

In [16]:
new_merged_plays.head(5)

,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event,Duration,Max_number_event
0,2017,399,1336,23449.0,2017-09-08 01:31:11.000,41.560001,54.279999,0.13,78.879997,192.050003,NaN,205,6
1,2017,399,1336,23449.0,2017-09-08 01:31:11.100,41.540001,54.139999,0.14,77.760002,190.759995,NaN,205,6
2,2017,399,1336,23449.0,2017-09-08 01:31:11.200,41.509998,54.000000,0.14,76.459999,189.559998,NaN,205,6
3,2017,399,1336,23449.0,2017-09-08 01:31:11.300,41.490002,53.860001,0.14,75.800003,188.449997,NaN,205,6
4,2017,399,1336,23449.0,2017-09-08 01:31:11.400,41.470001,53.709999,0.15,75.769997,187.589996,NaN,205,6


In [17]:
"""Regarder s'il y a tjs un snapp ball à chaque partie"""


"Regarder s'il y a tjs un snapp ball à chaque partie"

In [18]:
"""On va s'attaquer aux infos players pour chaque joueurs."""
player_role_punt = pd.read_csv("./data/play_player_role_data.csv")
player_role_punt.head(4)

,Season_Year,GameKey,PlayID,GSISID,Role
0,2017,414,188,33704,PDL2
1,2017,414,1107,33704,PDL2
2,2017,424,1113,33704,PDR3
3,2017,424,1454,33704,PLR2


In [19]:
final_merge = pd.merge(merged_plays, player_role_punt, on = ["Season_Year","GameKey","PlayID", "GSISID"], how = "inner")
final_merge.head(3)

,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event,Role
0,2017,399,1336,23534.0,2017-09-08 01:30:53.400,78.739998,29.820000,0.02,199.979996,168.630005,punt_play,P
1,2017,399,1336,23534.0,2017-09-08 01:30:53.500,78.750000,29.790001,0.03,199.009995,166.580002,NaN,P
2,2017,399,1336,23534.0,2017-09-08 01:30:53.600,78.769997,29.760000,0.04,198.029999,165.839996,NaN,P


In [20]:
print(new_merged_plays.shape)
final_merge.shape

(9422296, 13)


(7074107, 12)

In [21]:
print(new_merged_plays.shape)
final_merge.shape

(9422296, 13)


(7074107, 12)

In [22]:
groupe = final_merge.groupby (['Season_Year','GameKey', 'PlayID', 'GSISID'])["Time","Event"].count()
groupe.columns = ['Time_Player', 'Nb_event_played']
groupe.reset_index(inplace=True)
groupe.head(5)

,Season_Year,GameKey,PlayID,GSISID,Time_Player,Nb_event_played
0,2017,399,1336,23534.0,205,6
1,2017,399,1336,26035.0,205,6
2,2017,399,1336,27010.0,205,6
3,2017,399,1336,27442.0,205,6
4,2017,399,1336,28074.0,205,6


In [23]:
groupy = groupe.groupby(["Season_Year","GameKey","PlayID"])['Time_Player',"Nb_event_played"].max()
groupy.columns = ["Duration","Max_number_event"]
groupy.reset_index(inplace=True)
groupy.head(5)

,Season_Year,GameKey,PlayID,Duration,Max_number_event
0,2017,399,1336,205,6
1,2017,399,1776,862,10
2,2017,399,1884,971,8
3,2017,399,2572,212,6
4,2017,399,2712,663,11


In [24]:
select_player = pd.merge(groupe, groupy, on = ["Season_Year","GameKey","PlayID"], how = "inner")
print(select_player.shape[0])
select_player = select_player.loc[(select_player.Time_Player == select_player.Duration )]
print(select_player.shape[0])
del select_player['Time_Player']
del select_player['Nb_event_played']
select_player.head(4)#The shape are checked to be sure of changes

19884
19714


,Season_Year,GameKey,PlayID,GSISID,Duration,Max_number_event
0,2017,399,1336,23534.0,205,6
1,2017,399,1336,26035.0,205,6
2,2017,399,1336,27010.0,205,6
3,2017,399,1336,27442.0,205,6


In [25]:
test_22 = select_player.groupby(["Season_Year","GameKey","PlayID"])['GSISID'].count()
print(test_22.shape)
print(test_22.loc[test_22==22].shape)

(905,)
(854,)


Ok on se rend donc compte que l'on arrive au même résultat, à quelques résultats, ce sont donc des éjections ou autre sortie de joueurs. On va donc remplacer et on remplira les espaces manquants. Sinon on peut se limiter à ces plays avec le bon nombre de joueurs dans un premier temps. Je ne me limite pas pour le moment mais on pourra tenter de se limiter. 

In [26]:
plays_selected_players = pd.merge(final_merge, select_player, on = ["Season_Year","GameKey","PlayID", "GSISID"], how = 'inner')
print(final_merge.shape)
print(plays_selected_players.shape)
plays_selected_players.head(2)

(7074107, 12)
(7005607, 14)


,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event,Role,Duration,Max_number_event
0,2017,399,1336,23534.0,2017-09-08 01:30:53.400,78.739998,29.820000,0.02,199.979996,168.630005,punt_play,P,205,6
1,2017,399,1336,23534.0,2017-09-08 01:30:53.500,78.750000,29.790001,0.03,199.009995,166.580002,NaN,P,205,6


Pour trouver le sens des positions de chaque joueur : voir word explanations en annexe, dernière page

In [27]:
"""We make sure that it is sorted according to games and time, normally it's the case since the beginning"""
plays_selected_players.sort_values (['Season_Year','GameKey', 'PlayID', 'GSISID', 'Time'], inplace = True)

### Remarque sur les events : Il peut y avoir des fake punt plays, où l'équipe ne punt pas mais au contraire va jouer le touchdown ou le first down

# Missing Values

In [28]:
plays_selected_players.isnull().sum()

Season_Year               0
GameKey                   0
PlayID                    0
GSISID                    0
Time                      0
x                         0
y                         0
dis                       0
o                         0
dir                       0
Event               6858749
Role                      0
Duration                  0
Max_number_event          0
dtype: int64

In [29]:
plays_selected_players['Event'].fillna("NoEvent", inplace =True)

# Speed Computing

The main idea here is to put 2 columns with x and y of the following moment. Plus a column saying if the next moment belongs to the same play or not. All the plays will have at the end a missing value for the speed. 

In [30]:
plays_selected_players["previous_x"] = plays_selected_players['x'].shift(+1)
plays_selected_players["previous_y"] = plays_selected_players['y'].shift(+1)
plays_selected_players["SamePlayer_SamePlay"] = abs(plays_selected_players['GameKey']-\
                                                    plays_selected_players['GameKey'].shift(+1))+ \
                                                abs(plays_selected_players['PlayID']-\
                                                    plays_selected_players['PlayID'].shift(+1))+ \
                                                abs(plays_selected_players['GSISID']- \
                                                    plays_selected_players['GSISID'].shift(+1))
plays_selected_players.head(5)

,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event,Role,Duration,Max_number_event,previous_x,previous_y,SamePlayer_SamePlay
0,2017,399,1336,23534.0,2017-09-08 01:30:53.400,78.739998,29.820000,0.02,199.979996,168.630005,punt_play,P,205,6,NaN,NaN,NaN
1,2017,399,1336,23534.0,2017-09-08 01:30:53.500,78.750000,29.790001,0.03,199.009995,166.580002,NoEvent,P,205,6,78.739998,29.820000,0.0
2,2017,399,1336,23534.0,2017-09-08 01:30:53.600,78.769997,29.760000,0.04,198.029999,165.839996,NoEvent,P,205,6,78.750000,29.790001,0.0
3,2017,399,1336,23534.0,2017-09-08 01:30:53.700,78.790001,29.719999,0.04,196.990005,167.820007,NoEvent,P,205,6,78.769997,29.760000,0.0
4,2017,399,1336,23534.0,2017-09-08 01:30:53.800,78.800003,29.690001,0.03,195.960007,173.889999,NoEvent,P,205,6,78.790001,29.719999,0.0


In [31]:
def compute_speed(x):
    """This lambda function is supposed to compute the speed of a player in m/s"""
    if x["SamePlayer_SamePlay"]==0:
        return math.sqrt((x["previous_x"]-x["x"])**2+(x["previous_y"]-x["y"])**2)*10 #The 10 factor is supposed to counterbalance the 
                                                                            #time factor.
    else : 
        return None

In [32]:
time1 = time.time()
plays_selected_players["speed"] = plays_selected_players.apply(lambda x: compute_speed(x), axis= 1)
print((time.time()-time1)/60)

4.906651794910431


In [33]:
"""We have to fill the Nan on the speed column, so let's put the next speed"""
plays_selected_players["next_speed"] = plays_selected_players['speed'].shift(-1)

In [34]:
plays_selected_players.isnull().sum()

Season_Year                0
GameKey                    0
PlayID                     0
GSISID                     0
Time                       0
x                          0
y                          0
dis                        0
o                          0
dir                        0
Event                      0
Role                       0
Duration                   0
Max_number_event           0
previous_x                 1
previous_y                 1
SamePlayer_SamePlay        1
speed                  19714
next_speed             19714
dtype: int64

In [35]:
plays_selected_players['speed'].fillna(plays_selected_players['next_speed'], inplace=True)
plays_selected_players.isnull().sum()

Season_Year                0
GameKey                    0
PlayID                     0
GSISID                     0
Time                       0
x                          0
y                          0
dis                        0
o                          0
dir                        0
Event                      0
Role                       0
Duration                   0
Max_number_event           0
previous_x                 1
previous_y                 1
SamePlayer_SamePlay        1
speed                      0
next_speed             19714
dtype: int64

In [36]:
plays_selected_players.speed.max()

551.7051897270546

Il y a une anomalie sur cette donnée, soit une des données est fausse soit on a un cas limite pour le calcul de la vitesse qui nous a échappé. Il faut regarder de plus près. Même Usain Bolt ne fait pas 550m/S

In [37]:
plays_selected_players.iloc[:-1,:].speed.max()

551.7051897270546

Ce n'est pas la dernière valeur qui pose problème. A regarder demain.

In [38]:
plays_selected_players.speed.median()

1.2083227823231155

On va regarder les plays dont les vitesses instantanées sont importantes. 

In [39]:
plays_selected_players.loc[(plays_selected_players.speed>=13)|(plays_selected_players.shift(-1).speed>=13)|(plays_selected_players.shift(1).speed>=13)]

,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event,Role,Duration,Max_number_event,previous_x,previous_y,SamePlayer_SamePlay,speed,next_speed
4317,2017,399,1336,33947.0,2017-09-08 01:30:54.600,58.490002,-1.520000,0.25,272.559998,348.299988,NoEvent,PDL3,205,6,58.549999,-1.770000,0.0,2.570986,16.904733
4318,2017,399,1336,33947.0,2017-09-08 01:30:54.700,58.450001,0.170000,1.69,271.890015,348.549988,NoEvent,PDL3,205,6,58.490002,-1.520000,0.0,16.904733,0.499992
4319,2017,399,1336,33947.0,2017-09-08 01:30:54.800,58.400002,0.170000,0.06,271.040009,348.760010,NoEvent,PDL3,205,6,58.450001,0.170000,0.0,0.499992,0.900002
61544,2017,399,2712,32580.0,2017-09-08 02:40:00.600,40.779999,53.169998,0.53,350.540009,56.459999,NoEvent,GL,663,11,40.250000,53.169998,0.0,5.299988,19.358223
61545,2017,399,2712,32580.0,2017-09-08 02:40:00.700,41.349998,55.020000,1.94,351.459991,59.500000,NoEvent,GL,663,11,40.779999,53.169998,0.0,19.358223,6.964190
61546,2017,399,2712,32580.0,2017-09-08 02:40:00.800,41.939999,55.389999,0.70,352.160004,62.740002,NoEvent,GL,663,11,41.349998,55.020000,0.0,6.964190,7.083087
66366,2017,399,2943,32580.0,2017-09-08 02:50:48.600,64.750000,0.170000,0.49,6.440000,126.029999,punt,GR,124,6,64.260002,0.170000,0.0,4.899979,19.313199
66367,2017,399,2943,32580.0,2017-09-08 02:50:48.700,65.269997,-1.690000,1.93,2.790000,122.070000,NoEvent,GR,124,6,64.750000,0.170000,0.0,19.313199,6.603036
66368,2017,399,2943,32580.0,2017-09-08 02:50:48.800,65.809998,-2.070000,0.65,358.890015,118.300003,NoEvent,GR,124,6,65.269997,-1.690000,0.0,6.603036,6.500046
66383,2017,399,2943,32580.0,2017-09-08 02:50:50.300,75.199997,-1.560000,0.73,334.239990,77.239998,NoEvent,GR,124,6,74.500000,-1.770000,0.0,7.308185,18.700292


In [40]:
plays_selected_players.loc[(plays_selected_players.speed>=13)&(((plays_selected_players.shift(-1).speed<=10)&(plays_selected_players.shift(-2).speed<=10))|((plays_selected_players.shift(+1).speed<=10)&(plays_selected_players.shift(+2).speed<=10)))].shape

(760, 19)

In [41]:
plays_selected_players['RollingMean'] = (1/4.0)*(plays_selected_players.shift(-1)['speed'] +plays_selected_players.shift(+1)['speed']+plays_selected_players.shift(-2)['speed'] +plays_selected_players.shift(+2)['speed'])

In [47]:
plays_selected_players.loc[(plays_selected_players.speed>=13)&(((plays_selected_players.shift(-1).speed<=10)&(plays_selected_players.shift(-2).speed<=10))|((plays_selected_players.shift(+1).speed<=10)&(plays_selected_players.shift(+2).speed<=10))), 'speed'] = plays_selected_players['RollingMean']
#sub_plays_selected_players['speed'] = sub_plays_selected_players['RollingMean']


,Season_Year,GameKey,PlayID,GSISID,Time,x,y,dis,o,dir,Event,Role,Duration,Max_number_event,previous_x,previous_y,SamePlayer_SamePlay,speed,next_speed,RollingMean
4318,2017,399,1336,33947.0,2017-09-08 01:30:54.700,58.450001,0.17,1.69,271.890015,348.549988,NoEvent,PDL3,205,6,58.490002,-1.520000,0.0,1.654655,0.499992,1.654655
61545,2017,399,2712,32580.0,2017-09-08 02:40:00.700,41.349998,55.02,1.94,351.459991,59.500000,NoEvent,GL,663,11,40.779999,53.169998,0.0,6.086816,6.964190,6.086816


In [48]:
plays_selected_players.loc[(plays_selected_players.speed>=13)&(((plays_selected_players.shift(-1).speed<=10)&(plays_selected_players.shift(-2).speed<=10))|((plays_selected_players.shift(+1).speed<=10)&(plays_selected_players.shift(+2).speed<=10)))].shape

(31, 20)

In [53]:
plays_selected_players.loc[(plays_selected_players.speed>18),'speed'] = 10 # a typical high speed

In [54]:
plays_selected_players.isnull().sum()

Season_Year                0
GameKey                    0
PlayID                     0
GSISID                     0
Time                       0
x                          0
y                          0
dis                        0
o                          0
dir                        0
Event                      0
Role                       0
Duration                   0
Max_number_event           0
previous_x                 1
previous_y                 1
SamePlayer_SamePlay        1
speed                      0
next_speed             19714
RollingMean                4
dtype: int64

Speed and dir are perfectly clean now and seem more interpretable.

In [55]:
plays_selected_players.to_csv("./data/created/NGS-2017-reg-wk1-6_speed&role.csv", index = False, sep = ";", decimal = ".")